In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

In [2]:
# Import libraries
import pandas as pd
import os
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType, ArrayType, FloatType
import re
import string
from pyspark.sql.functions import udf, col, size, lit, explode, isnan, when, count, min, max, struct, posexplode
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, IndexToString, StringIndexer, VectorIndexer, CountVectorizer
from collections import Counter
import networkx as nx
import nltk
from nltk.corpus import words
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.models.ldamulticore import LdaMulticore
import matplotlib.pyplot as plt
%matplotlib inline
import openai
import json
import ast
from igraph import *
import datashader as ds
import datashader.transfer_functions as tf
from datashader.layout import random_layout, circular_layout, forceatlas2_layout
from datashader.bundling import connect_edges, hammer_bundle
import nlp
from textblob import TextBlob
import numpy as np
pd.set_option("display.max_columns", None)

In [3]:
# Initialize spark
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
spark = SparkSession.builder.getOrCreate()

22/08/22 08:00:54 WARN Utils: Your hostname, computer resolves to a loopback address: 127.0.1.1; using 192.168.1.159 instead (on interface wlp0s20f3)
22/08/22 08:00:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/08/22 08:00:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/22 08:00:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/22 08:00:55 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/08/22 08:00:55 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/08/22 08:00:55 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/08/22 08:0

In [4]:
# Read
df = spark.read.json('/home/dataguy/data/us_news_tweets.json').drop('_type', 'cashtags', 'conversationId', 'id', 'inReplyToTweetId', 'media', 'outLinks', 'renderedContent', 'sourceUrl', 'tcooutlinks', 'url')
df1 = df.withColumn('username', col('user.username')).withColumn('country', col('place.country')).withColumn('country_cd', col('place.countryCode'))
df2 = df1.withColumn('quoted', col('quotedTweet.user.username')).withColumn('mentions', col('mentionedUsers.username')).withColumn('reply_to', col('inReplyToUser.username')).drop('user', 'coordinates', 'place', 'quotedTweet', 'inReplyToUser', 'mentionedUsers', 'renderedContent', 'id', 'media', 'outlinks', '_type', 'cashtags', 'conversationId', 'inReplyToTweetId', 'source', 'sourceUrl', 'sourceLabel', 'tcooutlinks', 'url', 'country', 'lang', 'retweetedTweet')
df3 = df2.filter("country == 'United States'").toPandas()
df4 = df3.set_index(pd.DatetimeIndex(df3['date'])).drop('date', axis = 1)

22/08/22 08:01:04 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
df2.show(5)

+--------------------+--------------------+--------------------+---------+----------+----------+------------+---------------+----------+------+--------+--------+
|             content|                date|            hashtags|likeCount|quoteCount|replyCount|retweetCount|       username|country_cd|quoted|mentions|reply_to|
+--------------------+--------------------+--------------------+---------+----------+----------+------------+---------------+----------+------+--------+--------+
|Pagaya Technologi...|2022-08-16T10:41:...|[news, SeekingAlpha]|        0|         0|         0|           0|JacekWierzbicki|      null|  null|    null|    null|
|Nasdaq, S&amp;P, ...|2022-08-16T10:41:...|[news, SeekingAlpha]|        0|         0|         0|           0|JacekWierzbicki|      null|  null|    null|    null|
|Kaixin Auto stock...|2022-08-16T10:41:...|[news, SeekingAlpha]|        0|         0|         0|           0|JacekWierzbicki|      null|  null|    null|    null|
|Sea Non-GAAP EPS ...|2022-0

In [6]:
df2.filter(~df2.quoted.isNull()).show()

+--------------------+--------------------+--------------------+---------+----------+----------+------------+---------------+----------+---------------+--------------------+--------+
|             content|                date|            hashtags|likeCount|quoteCount|replyCount|retweetCount|       username|country_cd|         quoted|            mentions|reply_to|
+--------------------+--------------------+--------------------+---------+----------+----------+------------+---------------+----------+---------------+--------------------+--------+
|This perpetrator ...|2022-08-16T10:32:...|              [news]|        1|         0|         0|           1|        DHMOban|      null|9NewsQueensland|                null|    null|
|#NEWS #JAPANNEWS ...|2022-08-16T10:27:...|   [NEWS, JAPANNEWS]|        0|         0|         0|           0|       OSSORNUM|      null|         zlj517|                null|    null|
|At first I though...|2022-08-16T10:22:...|[SnoopDogg, Snoop...|        0|         0|

In [7]:
df2.printSchema()

root
 |-- content: string (nullable = true)
 |-- date: string (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- likeCount: long (nullable = true)
 |-- quoteCount: long (nullable = true)
 |-- replyCount: long (nullable = true)
 |-- retweetCount: long (nullable = true)
 |-- username: string (nullable = true)
 |-- country_cd: string (nullable = true)
 |-- quoted: string (nullable = true)
 |-- mentions: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- reply_to: string (nullable = true)



In [8]:
df2.count()

2877354

In [9]:
users = df2.select('username').distinct().collect()
quoted = df2.select('quoted').distinct().collect()
mentions = df2.select('mentions').distinct().collect()

In [10]:
mentions[0:10]

[Row(mentions=['baltimoresun']),
 Row(mentions=['pakstockexgltd']),
 Row(mentions=['theheraldsun']),
 Row(mentions=['POTUS', 'SkyNews', 'MSNBC', 'abcnews', 'FoxNews', 'cnnbrk', 'BBCBreaking', 'BBCNews', 'BBCWorld']),
 Row(mentions=['SociumJob']),
 Row(mentions=['lingerie_addict']),
 Row(mentions=['BUAgroup', 'MBuhari']),
 Row(mentions=['rshaveddinov']),
 Row(mentions=['RaapBot', 'rtItBot', 'Retweelgend', 'AppleMusicRT', 'DistroKid']),
 Row(mentions=['FUFADrum'])]

In [11]:
mnts = []

# Iterate mentions
for x in mentions:
    
    if x[0]:
        
        for y in x[0]:
        
            mnts.append(y)

In [12]:
mnts[0:5]

['baltimoresun', 'pakstockexgltd', 'theheraldsun', 'POTUS', 'SkyNews']

In [13]:
nodes = [x[0] for x in users] + [y[0] for y in quoted] + [i for i in set(mnts)]
nods = [i for i in set(nodes)]

In [14]:
len(nodes)

196820

In [15]:
len(nods)

184121

In [16]:
pd.DataFrame(nods, columns = ['username']).to_csv('/home/dataguy/data/us_news_nodes.csv', index = False)

In [17]:
edges = []

# Iterate data to build network connections
for x in df2.select('username', 'quoted', 'mentions', 'reply_to').collect():
    
    if x[1]:
        
        edges.append((x[0], x[1], 'quote'))
        
    if x[2]:
        
        for y in x[2]:
            
            edges.append((x[0], y, 'mention'))
            
    if x[3]:
        
        edges.append((x[0], x[3], 'reply'))

In [18]:
edges[0:10]

[('E_4_Jupiter', 'Frontex', 'mention'),
 ('coryboatright', 'HomeServicesPM', 'mention'),
 ('coryboatright', 'fortunebuilders', 'mention'),
 ('coryboatright', 'RoyalNewsfeed', 'mention'),
 ('ExecuTeamUSA', 'interney', 'mention'),
 ('ExecuTeamUSA', 'nbcchicago', 'mention'),
 ('ExecuTeamUSA', 'craigiskowitz', 'mention'),
 ('DHMOban', '9NewsQueensland', 'mention'),
 ('DHMOban', '9NewsQueensland', 'reply'),
 ('LeavSteinberg', 'nj1015', 'mention')]

In [19]:
edgs = pd.DataFrame(edges, columns = ['source', 'target', 'type'])

In [20]:
e1 = edgs.groupby(['source', 'target', 'type']).size().reset_index().rename({0: 'weight'}, axis = 1)

In [21]:
e2 = e1[e1['source'].str.strip() != e1['target'].str.strip()]

In [22]:
e2.sort_values('weight', ascending = False).head(n = 10)

,source,target,type,weight
182175,thehindupatrika,TheNileshDesai,mention,12667
73402,MetronixSA,TechCrunch,mention,4764
57905,Jessica_L_Hunt,baltimoresun,mention,3825
146201,hensleragency,nytimes,mention,3264
173877,rvp,DeepFriedCyber,quote,2573
46625,GiGiCaponePR,Complex,mention,2495
37238,EconomySquare,CraftyFin,mention,2351
83103,Othershawnlong,xdadevelopers,mention,2196
93683,Rodrick76496943,SportfusionApps,mention,1977
190603,ymediagroup,YudhvirJaswal,mention,1179


In [23]:
e2.shape[0]

185388

In [24]:
e2.to_csv('/home/dataguy/data/us_news_edges.csv', index = False)

In [25]:
# Filter out information on all users
users = pd.DataFrame(df.select('inReplyToUser.*').distinct().toPandas()).drop(['_type', 'descriptionUrls', 'id', 'label', 'linkTcourl', 'linkUrl', 'profileBannerUrl', 'profileImageUrl', 'url'], axis = 1)

In [40]:
users.head()

,created,description,displayname,favouritesCount,followersCount,friendsCount,listedCount,location,mediaCount,protected,rawDescription,statusesCount,username,verified
0,None,None,KGET 17 News,NaN,NaN,NaN,NaN,None,NaN,None,None,NaN,KGETnews,None
1,2015-04-28T14:49:11+00:00,Serving the residents of #MissoulaCounty since...,"Missoula County, Montana",859.0,2244.0,538.0,56.0,"Missoula, MT",1940.0,False,Serving the residents of #MissoulaCounty since...,5310.0,MissoulaCounty,False
2,2021-08-04T06:18:42+00:00,"Listening to music 🎶, creative, jigsaw puzzle 🧩",Hs. YH,2401.0,104.0,547.0,1.0,,1479.0,False,"Listening to music 🎶, creative, jigsaw puzzle 🧩",2575.0,Qiu79234655,False
3,2014-03-01T20:13:31+00:00,🎗️ “Always Look On The Bright Side Of Life.” —...,⚡️cyberalgo,1071.0,884.0,619.0,18.0,,72470.0,False,🎗️ “Always Look On The Bright Side Of Life.” —...,123551.0,cyberalgo,False
4,None,None,Ian Coles,NaN,NaN,NaN,NaN,None,NaN,None,None,NaN,i_coles,None


In [63]:
collected = df.select('mentionedUsers').distinct().collect()
drops = ['_type', 'descriptionUrls', 'id', 'label', 'linkTcourl', 'linkUrl', 'profileBannerUrl', 'profileImageUrl', 'url']

In [81]:
mentioned_users = []

for x in collected:
    
    if x['mentionedUsers']:
        
        for y in x['mentionedUsers']:
        
            mentioned_users.append(y)

In [90]:
mentioned_users = pd.DataFrame(mentioned_users, columns = [mentioned_users[0].__fields__]).drop(['_type', 'descriptionUrls', 'id', 'label', 'linkTcourl', 'linkUrl', 'profileBannerUrl', 'profileImageUrl', 'url'], axis = 1)

In [91]:
mentioned_users.head()

,created,description,displayname,favouritesCount,followersCount,friendsCount,listedCount,location,mediaCount,protected,rawDescription,statusesCount,username,verified
0,None,None,المصرية للاتصالات,None,None,None,None,None,None,None,None,None,telecomegypt,None
1,None,None,Deltek,None,None,None,None,None,None,None,None,None,Deltek,None
2,None,None,Christopher C. Cuomo,None,None,None,None,None,None,None,None,None,ChrisCuomo,None
3,None,None,GvonT,None,None,None,None,None,None,None,None,None,GvonTuch,None
4,None,None,Garland Nixon,None,None,None,None,None,None,None,None,None,GarlandNixon,None
